# 02 - Data Cleaning and Preprocessing

## Objectives
- Handle missing values appropriately
- Remove sparse features
- Encode categorical variables
- Prepare data for machine learning

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/Users/sararosati/Desktop/vscode-projects/Heritage-Housing/Heritage-housing/jupyter_notebooks'

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")
current_dir = os.getcwd()
current_dir

You set a new current directory


'/Users/sararosati/Desktop/vscode-projects/Heritage-Housing/Heritage-housing'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

Libraries imported successfully!


In [4]:
# Load the datasets
df = pd.read_csv('inputs/datasets/house_prices_records.csv')
inherited_df = pd.read_csv('inputs/datasets/inherited_houses.csv')

print(f"Main dataset shape: {df.shape}")
print(f"Inherited houses shape: {inherited_df.shape}")
print("\nDataset loaded successfully!")

Main dataset shape: (1460, 24)
Inherited houses shape: (4, 23)

Dataset loaded successfully!


## Section 1: Handling Missing Values

In [5]:
# Step 1: Drop features with >80% missing values
print("Features with >80% missing values:")
missing_percentage = (df.isnull().sum() / len(df)) * 100
sparse_features = missing_percentage[missing_percentage > 80]
print(sparse_features)

# Drop these sparse features
df_cleaned = df.drop(columns=sparse_features.index)

print(f"\nDataset shape after dropping sparse features: {df_cleaned.shape}")
print(f"Features removed: {list(sparse_features.index)}")

Features with >80% missing values:
EnclosedPorch    90.684932
WoodDeckSF       89.383562
dtype: float64

Dataset shape after dropping sparse features: (1460, 22)
Features removed: ['EnclosedPorch', 'WoodDeckSF']


## Step 2: Handle Remaining Missing Values